#  資料爬蟲與視覺化專案-國民平均所得換算與分析

# 專案

作者:陳靖
日期:2018/08/07

# 專案簡介

大家都知道每年台灣公布的經濟成長率長年為正數

但是相較之下的經濟感覺卻不是那麼明顯 

因為實際的經濟會受到物價與匯率的影響 

我的作業想要透過換算呈現真實的經濟 

將單純的數字透過換算成較真實且容易感受的圖表 

以下分為三個部分

1.蒐集資料

2.整理資料

3.運算資料與視覺化

# 蒐集資料

為了收集所需資料

會使用爬蟲軟體從以下網站蒐集資料並存檔

油價資料:quandl平台

國民平均所與物價指數資料:行政院主計處

外匯資料:中央銀行

蒐集的資料如下


| 年分 | 國民平均所得   |油價 |物價指數 |  美金匯率|
|------|------|------|------|------|
| xxxx  | xxxx  | xxxx  | xxxx  |xxxx  |
| xxxx  |xxxx  | xxxx  | xxxx  | xxxx  |
| xxxx  | xxxx  | xxxx  | xxxx  | xxxx  |

使用函式的原因是方便決定是否要提取這份資料

# 蒐集資料程式

In [ ]:
#取得歷史油價json檔
def get_oilprice():
    oil_url = 'https://www.quandl.com/api/v3/datasets/BP/CRUDE_OIL_PRICES.json?api_key=9cLNX2qEVMTZaRysiv7_'
    oil_r=requests.get(oil_url)
    oil_response = (oil_r.json())

    with open('oil_r.json','w',encoding='utf-8-sig') as f_oi:
        json.dump(oil_response,f_oi)

#取得歷史國民平均所得excel檔並轉成csv檔
def average_ni():
    average_NI= "https://www.dgbas.gov.tw/public/data/dgbas03/bs4/nis93/np02.xls"
    average= requests.get(average_NI)
 
    output = open('average.xls', 'wb')
    output.write(average.content)
    output.close()

    data_xls = pd.read_excel('average.xls', 'P2', index_col=None)
    data_xls.to_csv('average_ni', encoding='utf-8-sig')
    
#取得歷史匯率json檔並取出需要資料轉成csv檔
def exchange():
    url = 'https://quality.data.gov.tw/dq_download_json.php?nid=6563&md5_url=ea67c565f721b6920a1982599526ed0b'

    r=requests.get(url)
    response = (r.json())

    with open('exchange.json','w',encoding='utf-8-sig') as f_ex:
        json.dump(response,f_ex)

    with open('exchange.json','r',encoding='utf-8-sig') as f_exs:
        exchange=json.loads(f_exs.read())
        n=len(exchange)
        n1=n-1
        with open('exchange.csv','w',encoding='utf-8-sig') as fw:
            filewriter = csv.writer(fw, delimiter=',')
            filewriter.writerow(['YEAR','NTD:USD','JPY:USD','CNY:USD'])
            for i in range(0,n1):
                YEAR=exchange[i]['期間']
                NTD=exchange[i]['新台幣NTD/USD']
                JPY=exchange[i]['日圓JPY/USD']
                CNY=exchange[i]['人民幣CNY/USD']
                filewriter.writerow([YEAR,NTD,JPY,CNY])

#取得歷史物價excel檔並轉成csv檔
def price():
    average_price= "https://www.stat.gov.tw/public/data/dgbas03/bs3/inquire/cpispl.xls"
    average= requests.get(average_price)
 
    output = open('average_price.xls', 'wb')
    output.write(average.content)
    output.close()

    data_xls = pd.read_excel('average_price.xls', 'CPI', index_col=None)
    data_xls.to_csv('average_price', encoding='utf-8-sig')

# 整理資料

取得各種資料檔案後

將所需要的資料提成串列

再用pandas整合成需要的資料

# 整理資料程式

In [ ]:
#整理歷史平均收入成串列
average_ni()

list_average_ni_year=[]
list_average_ni_income=[]
with open('average_ni', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    headers = next(f)
    headers = next(f)
    for row in reader:
        year=re.search('\d+',row[0])
        income=re.search('\d+',row[8])
        if year!=None:
            list_average_ni_year.append(int(year.group())+1911)
        if income!=None:
            list_average_ni_income.append(int(income.group())/12)

#整理歷史物價成串列
price()
with open('average_price', 'r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        headers = next(f)
        headers = next(f)
        list_CPI_year=[]
        list_CPI_number=[]
        for row in reader:
            year=re.search('\d+',row[1])
            CPInumber=re.search('[0-9]+\.*[0-9]*',row[14])
            if year!=None and int(year.group())>40:
                list_CPI_year.append(int(year.group())+1911)
            if CPInumber!=None:
                list_CPI_number.append(float(row[14]))

#整理歷史匯率成串列        
exchange()
with open('exchange.csv', 'r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        headers = next(f)
        list_exchange_year=[]
        list_exchange_number=[]
        for row in reader:
            if row!=[]:
                list_exchange_year.append(int(row[0]))
                list_exchange_number.append(float(row[1]))
                
#整理歷史油價成串列
get_oilprice()
with open('oil_r.json','r',encoding='utf-8-sig') as f:
    oil=json.loads(f.read())
    n=len(oil["dataset"]["data"])
    n1=n-1
    list_oil_year=[]
    list_oil_ori=[]
    list_oil_priinf=[]
    for i in range(0,n1):
        oil_year=re.search('\d+',oil["dataset"]["data"][i][0])
        list_oil_year.append(int(oil_year.group()))
        list_oil_ori.append(float(oil["dataset"]["data"][i][1]))
        list_oil_priinf.append(float(oil["dataset"]["data"][i][2]))
        
#將收入資料轉換成二維表格
dict1 = {'year': list_average_ni_year,'average_ni':list_average_ni_income}
df1 = pd.DataFrame(dict1)

#將物價資料轉換成二維表格
dict2 = {'year': list_CPI_year,'CPI':list_CPI_number}
df2 = pd.DataFrame(dict2)

#將匯率資料轉換成二維表格
dict3 = {'year': list_exchange_year,'Exchange':list_exchange_number}
df3 = pd.DataFrame(dict3)

#將油價資料轉換成二維表格
dict4 = {'year': list_oil_year,'oil_ori':list_oil_ori,'oil_priinf':list_oil_priinf}
df4 = pd.DataFrame(dict4)

#合併收入與物價資料，將資料不存在的年份去除
df_g1 = pd.merge(df1,df2, on='year', how='outer')
data1=df_g1.dropna()

#合併收入與匯率資料，將資料不存在的年份去除，紀錄最近期匯率
df_g2 = pd.merge(df1,df3, on='year', how='outer')
data2=df_g2.dropna()
now=data2.iloc[-1]
now_exchange=now[2]

#合併收入與油價資料，將資料不存在的年份去除，紀錄最近期油價
df_g3 = pd.merge(df1,df4, on='year', how='outer')
data3=df_g3.dropna()
now=data3.iloc[-1]
now_oil=now[2]

# 運算資料與視覺化

有了整理好的資料後

就能開始運算

這邊會先將歷史國民平均收入除以12換算成月薪比較易懂

畫出第一張圖呈現台灣國民平均月薪所得的變化

1.連結國民平均月薪所得與物價的關聯

    將物價資料導入後換算出通膨後所得並與原始資料做比較

    畫出第二張圖呈現台灣國民平均月薪所得與通膨的變化
    
2.連結國民平均月薪所得與物價的關聯

    因為台灣是出口導向貿易的國家
    
    出口外匯佔台灣經濟很大一部分收入
    
    所以這邊美金將匯率導入

    換算出匯率改變的情況下所得並與原始資料做比較

    畫出第三張圖呈現台灣國民平均月薪所得與匯率的變化
    
 3.連結國民平均月薪所得與油價的關聯

    石油左右了世界的經濟也主導了台灣的經濟與生產
    
    而且因為美金是交易石油的主要貨幣
    
    所以這邊將美金匯率導入結合歷史油價

    換算出匯率與油價改變的情況下所得並與原始資料做比較

    畫出第三張圖呈現台灣國民平均月薪所得與石油價格的變化



In [ ]:
#第一張圖表
plt.plot(list_average_ni_year,list_average_ni_income,label='原始國民平均月薪收入',color='blue',linewidth=2.0)
plt.title(u'國民平均月所得')
plt.xlabel(u'年(西元)')
plt.ylabel(u'       金額(新台幣)       ',rotation=360)
plt.legend()
plt.show()

#第二張圖表
plt.plot(list_average_ni_year,list_average_ni_income,label='原始國民平均月薪收入',color='blue',linewidth=2.0)
plt.plot(data1['year'],data1['average_ni']*100/data1['CPI'],label='物價調整國民平均月薪收入',color='red',linewidth=2.0)
plt.title(u'國民平均月所得(物價調整)')
plt.xlabel(u'年(西元)')
plt.ylabel(u'       金額(新台幣)       ',rotation=360)
plt.legend()
plt.show()

#第三張圖表 將歷年平均月所得以現在匯率換成美金 再用歷史匯率換回台幣
plt.plot(list_average_ni_year,list_average_ni_income,label='原始國民平均月薪收入',color='blue',linewidth=2.0)
plt.plot(data2['year'],data2['average_ni']*data2['Exchange']/now_exchange,label='匯率調整國民平均月薪收入',color='green',linewidth=2.0)
plt.title(u'國民平均月所得(匯率)')
plt.xlabel(u'年(西元)')
plt.ylabel(u'       金額(新台幣)       ',rotation=360)
plt.legend()
plt.show()

#第四張圖表 將歷年平均月所得以現在油價匯率換成等價石油 再用歷史匯率與油價換回台幣
plt.plot(list_average_ni_year,list_average_ni_income,label='原始國民平均月薪收入',color='blue',linewidth=2.0)
plt.plot(data3['year'],data3['average_ni']*now_oil*now_exchange/(data2['Exchange']*data3['oil_ori']),label='油價匯率調整國民平均月薪收入',color='pink',linewidth=2.0)
plt.plot(data3['year'],data3['oil_ori']*1000,label='油價',color='yellow',linewidth=2.0)
plt.title(u'國民平均月所得')
plt.xlabel(u'年(西元)')
plt.ylabel(u'       金額(新台幣)       ',rotation=360)
plt.legend()
plt.show()

# 結論

原始國民平均月薪

![Image1](https://imgur.com/j8ZvEft.png)

由圖一可以看出就純金額當量上

台灣的經濟從過去到現在有顯著的成長

特別是1980到2000年的增額幅度大

正好符合大眾認知的經濟奇蹟時期

![Image2](https://imgur.com/G9uNmpp.png)

由圖二可以看出就算在物價換算後

台灣的經濟仍從過去到現在有顯著的成長

但增額幅度較原始經濟數據小

反映經濟成長一定伴隨著通膨

而1970到2008年

調整線與原始線呈定比成長

反映此時的物價通膨是反映經濟成長的良性通膨

只是之後差距漸小

指出物價成長的幅度已經追上經濟成長

顯的經濟感覺起來反而沒有這麼良好

![Image3](https://imgur.com/TjjfTrU.png)

由圖三可以看出就算台幣對美金的匯率相當穩定

換算起來的差異範圍不大

匯率本身的起伏也不大

反映台灣匯率是由央行控制

目的在於穩定經濟而非成長

![Image4](https://imgur.com/DYUFpcK.png)

由圖四跟其他圖片呈現較特別的結果

此總換算成呈現出油價的變化與匯率變動

因為石油多用美金交易

而台灣最大宗的進口商品也是石油

可以看出從1980開始

台灣取得一桶石油的成本比較同時收入相對低廉

因此相對生產成本、運輸成本、能源成本都較低

相對收入在1970開始到2005非常高

也符合台灣經濟起飛時期

可知石油價格對於台灣的生產及出口貿易有很大的影響

# 後續

本次分析採用的數據為國民平均所得

在數據上難以反映真實收入的中位數

之後如想使分析更貼近大眾應改用所的中位數

而換算的方法亦可改進

因為物價指數是持續況張的

單用一項指示難以反應會發展出現的新型娛樂、食品、設備、建築工法及電子產品等等

應能用改用更加反映真實數據的換算方法

而油價與匯率轉換部分

應在再考慮單一油價對影響的影響和並用進口量做加權變換

才能反映真正進口石油對台灣的影響